## Busca Vetorial com Elasticsearch

Uma das novas funcionalidades do **Elasticsearch** é a capacidade de operar com **vetores**, funcionando como um **banco vetorial**. Essa funcionalidade ganhou destaque com o aumento da utilização de **transformers**, redes neurais capazes de representar um texto como um vetor numérico.

Essa nova forma de representar textos permite identificar conteúdos com **significados semelhantes**, mesmo quando são utilizadas **palavras diferentes** — algo que algoritmos tradicionais, como **TF-IDF** e **Okapi BM25**, não conseguem fazer com a mesma eficiência.

Neste notebook, iremos:
- **Salvar os vetores das frases**, e
- **Realizar buscas utilizando o vetor da frase de consulta**.

Para gerar os vetores das frases, será utilizado  o modelo [BAAI/bge-m3](https://huggingface.co/BAAI/bge-m3) que gera um vetor de 1024 dimensao

In [3]:
#O texto abaixo é a letra de uma música

letra = "Era um cômodo, incômodo, sujo como dragão de komodo  \n  Úmido, eu homem da casa aos seis anos  \n  Mofo no canto, todo, Tv, engodo, pronto pro lodo  \n  Tímido, porra, somos reis, mano  \n  Olhos são eletrodos, sério, topo, trombo corvos  \n  Num cemitério de sonhos, graças a leis, planos  \n  Troco de jogo, vendo roubos, pus a cabeça a prêmio  \n  Ingênuo, colhi sorrisos e falei -- vamos!  \n  É um novo tempo, momento pro novo, ao sabor do vento  \n  Me movo pelo solo onde reinamos  \n  Pondo pontos finais na dor, como doril, anador  \n  Somos a luz do senhor, pode crer, tamo  \n  Construindo, suponho não, creio, meto a mão  \n  Em meio à escuridão, pronto, acertamos  \n  Nosso sorriso sereno hoje é o veneno  \n  Pra quem trouxe tanto ódio pr'onde deitamos  \n    \n  Quem costuma vir de onde eu sou  \n  Às vezes não tem motivos pra  \n  Seguir  \n  Então levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda  \n  Mas eu sei que vai, que o sonho te traz  \n  Coisas que te faz  \n  Prosseguir  \n  Vai, levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda, vai  \n    \n  Irmão, você não percebeu que você  \n  É o único representante do seu sonho na face da Terra?  \n  Se isso não fizer você correr, chapa  \n  Eu não sei o que vai  \n    \n  Eu sei, sei, cansa  \n  Quem morre ao fim do mês  \n  Nossa grana ou nossa esperança?  \n  Delírio é  \n  Equilíbrio entre nosso martírio e nossa fé  \n  Foi foda contar migalha nos escombros  \n  Lona preta esticada, enxada no ombro e nada vim  \n  Nada enfim, recria  \n  Sozim, com alma cheia de mágoa e as panela vazia  \n  Sonho imundo  \n  Só água na geladeira e eu querendo salvar o mundo  \n  No fundo é tipo David Blaine, mãe assume, pai some  \n  De costume, no máximo é um sobrenome  \n  Sou terror dos clone  \n  Esses boy conhece Marx, nóiz conhece a fome  \n  Então cerre os punhos, sorria  \n  E jamais volte pra sua quebrada de mão e mente vazia  \n    \n  Quem costuma vir de onde eu sou  \n  Às vezes não tem motivos pra  \n  Seguir  \n  Então levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda  \n  Mas eu sei que vai, que o sonho te traz  \n  Coisas que te faz  \n  Prosseguir  \n  Então levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda  \n    \n  Somos maior, nos basta só  \n  Sonhar, seguir!"


sentencas = letra.split("\n")
sentencas = [sentenca.strip() for sentenca in sentencas]

In [4]:
# Criando um indice. Neste caso, é necessário configurar o mapeamento


# Criando o conector com elasticsearch
# Modificar a variável PASSWORD para a senha configurado no .env
from elasticsearch import Elasticsearch

HOST = 'http://localhost:9200'
USERNAME = 'elastic'
PASSWORD = 'test10'

client = Elasticsearch(
    HOST,
    basic_auth=(USERNAME,PASSWORD)
)



In [5]:

mapping = mappings = {
    "properties":{
        "vetor_sentenca":{
            "type":"dense_vector",
            "dims":1024, #dimensao 
            "similarity":"cosine", #algoritmo de similaridade
            "index":True
        },
        "sentenca":{
            "type":"text"
        }
    }
}

In [6]:
INDEX_LETRA_DENSE = 'vetor_letra_sentenca'

In [7]:
#criando o indice
if client.indices.exists(index=INDEX_LETRA_DENSE):
    client.indices.delete(index=INDEX_LETRA_DENSE)
client.indices.create(index=INDEX_LETRA_DENSE, body={"mappings":mappings})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vetor_letra_sentenca'})

In [1]:
#Para utilizar o modelo de linguagem, é necessário instalar a biblioteca sentence-transformers 
#pip install sentence-transformers
from sentence_transformers import SentenceTransformer

C:\Users\spguilhermem\Documents\Pessoal\4_database_in_one_lecture\4databaseenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Carregando o modelo de linguagem  - Pode demorar um pouco pois o modelo é grande
model = SentenceTransformer('BAAI/bge-m3')

In [8]:
#mostrando como o modelo representa uma sentença
sentencas_vetores = list()
for sentenca in sentencas:
    vetor_sentenca = model.encode(sentenca).tolist()  # Convertendo para lista
    doc = {
        "vetor_sentenca": vetor_sentenca,
        "sentenca": sentenca
    }
    sentencas_vetores.append(doc)


In [9]:
print(f"Mostrando a sentença: {sentencas_vetores[0]['sentenca']} \n")
print(f"Mostrando o vetor correspondente: {sentencas_vetores[0]['vetor_sentenca']}")

Mostrando a sentença: Era um cômodo, incômodo, sujo como dragão de komodo 

Mostrando o vetor correspondente: [0.019787155091762543, 0.014598661102354527, -0.04936213046312332, -0.05649177357554436, -0.024416903033852577, -0.042541489005088806, -0.0033683052752166986, -0.014608055353164673, -0.02200082130730152, 0.0142286391928792, 0.030154000967741013, 0.007165569346398115, -0.04114692658185959, -0.05991034954786301, 0.04446803033351898, 0.02807674929499626, 0.06614116579294205, -0.020416485145688057, -0.014391413889825344, 0.011879470199346542, -0.00965846236795187, 0.028665831312537193, 0.0019323170417919755, 0.041535891592502594, -0.05748212710022926, 0.03252630680799484, -0.008169153705239296, 0.04146762192249298, 0.013093501329421997, -0.015113678760826588, 0.009628522209823132, 0.007235199678689241, -0.0050852117128670216, 0.010399368591606617, -0.008206138387322426, -0.034561727195978165, 0.019113944843411446, 0.031566374003887177, -0.005912010092288256, 0.010761945508420467, -

In [10]:
#Inserindo os vetores no índice
for doc in sentencas_vetores:
    client.index(index=INDEX_LETRA_DENSE, body=doc)

In [18]:
#Pesquisando a sentença mais similar
query_sentenca = "falece no termino"



Essa sentença foi escolhida pois não existe a palavra "falece" ou "termino"

In [21]:
#Pesquisa termos semelhantes a query usando o modelo tradicional do elastisearch
query = {
    "query": {
        "match": {
            "sentenca": query_sentenca
        }
    },
    "_source":['sentenca']
}

response = client.search(index=INDEX_LETRA_DENSE, body=query)

In [22]:
response.body

{'took': 35,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4, 'relation': 'eq'},
  'max_score': 2.4937928,
  'hits': [{'_index': 'vetor_letra_sentenca',
    '_id': 'kXn0UJcBCLEl5ISoK52u',
    '_score': 2.4937928,
    '_source': {'sentenca': 'De costume, no máximo é um sobrenome'}},
   {'_index': 'vetor_letra_sentenca',
    '_id': 'Y3n0UJcBCLEl5ISoJp1D',
    '_score': 2.2089686,
    '_source': {'sentenca': 'Mofo no canto, todo, Tv, engodo, pronto pro lodo'}},
   {'_index': 'vetor_letra_sentenca',
    '_id': 'i3n0UJcBCLEl5ISoK50Q',
    '_score': 2.2089686,
    '_source': {'sentenca': 'Lona preta esticada, enxada no ombro e nada vim'}},
   {'_index': 'vetor_letra_sentenca',
    '_id': 'kHn0UJcBCLEl5ISoK52Q',
    '_score': 2.0896366,
    '_source': {'sentenca': 'No fundo é tipo David Blaine, mãe assume, pai some'}}]}}

In [23]:
#Criando a pesquisa com o modelo de linguagem e pesquisando vetor semelhante
query_vector = model.encode(query_sentenca)

In [26]:
#Pesquisa de similaridade
query_vector = model.encode(query_sentenca)
query = {
     "_source":['sentenca'],
    "knn":{
       "field":"vetor_sentenca",
       "query_vector":query_vector,
       "k":3,
       "num_candidates":100

    }
    
}

response = client.search(index=INDEX_LETRA_DENSE, body=query)

In [27]:
response.body

{'took': 32,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 0.8481451,
  'hits': [{'_index': 'vetor_letra_sentenca',
    '_id': 'hnn0UJcBCLEl5ISoKp2K',
    '_score': 0.8481451,
    '_source': {'sentenca': 'Quem morre ao fim do mês'}},
   {'_index': 'vetor_letra_sentenca',
    '_id': 'jHn0UJcBCLEl5ISoK50x',
    '_score': 0.74912214,
    '_source': {'sentenca': 'Nada enfim, recria'}},
   {'_index': 'vetor_letra_sentenca',
    '_id': 'cXn0UJcBCLEl5ISoKJ1R',
    '_score': 0.7374246,
    '_source': {'sentenca': ''}}]}}

Observe que neste caso, a busca vetorial possui um melhor resultado pois a frase e o primeiro objeto retornado tem significado semântico semelhante